In [4]:
import sympy as sy
from sympy import sin, cos, pi, sqrt
import math

q = sy.Matrix(sy.MatrixSymbol('q',4, 1))
l1, l2, l3, l4 = sy.symbols('l1, l2, l3, l4')

def HTM(theta, l):
    return sy.Matrix([
        [cos(theta), -sin(theta), l],
        [sin(theta), cos(theta), 0],
        [0, 0, 1]
    ])

Ts = [
    HTM(q[0, 0], 0),
    HTM(q[1, 0], l1),
    HTM(q[2, 0], l2),
    HTM(q[3, 0], l3),
    HTM(0, l4)
]

for i, T in enumerate(Ts):
    if i == 0:
        T_abs = [T]
    else:
        T_abs.append(T_abs[i-1] @ T)
    
    print(T_abs[-1].subs([(q[0,0], 0),(q[1,0], 0),(q[2,0], 0),(q[3,0], 0)]))

os = [T[0:2, 2:3] for T in T_abs]
Rxs = [T[0:2, 0:1] for T in T_abs]
Rys = [T[0:2, 1:2] for T in T_abs]

Jos = [o.jacobian(q) for o in os]
JRxs = [r.jacobian(q) for r in Rxs]
JRys = [r.jacobian(q) for r in Rys]


t = sy.Symbol("t")
q1 = sy.Function("q1")
q2 = sy.Function("q2")
q3 = sy.Function("q3")
q4 = sy.Function("q4")

dq = sy.Matrix(sy.MatrixSymbol('dq', 4, 1))

T_abs_ = []
for T in T_abs:
    T_ = T.subs([
        (q[0,0], q1(t)),
        (q[1,0], q2(t)),
        (q[2,0], q3(t)),
        (q[3,0], q4(t)),
    ])
    T_abs_.append(T_)


os_ = [T[0:2, 2:3] for T in T_abs_]
Rxs_ = [T[0:2, 0:1] for T in T_abs_]
Rys_ = [T[0:2, 1:2] for T in T_abs_]


q_ = sy.Matrix([
    [q1(t)],
    [q2(t)],
    [q3(t)],
    [q4(t)],
])
Jos_ = [o.jacobian(q_) for o in os_]
JRxs_ = [r.jacobian(q_) for r in Rxs_]
JRys_ = [r.jacobian(q_) for r in Rys_]

Jos_dot_ = [sy.diff(J, t) for J in Jos_]
JRxs_dot_ = [sy.diff(J, t) for J in JRxs_]
JRys_dot_ = [sy.diff(J, t) for J in JRys_]


Jos_dot = []
JRxs_dot = []
JRys_dot = []

for Js, newJs in zip((Jos_dot_, JRxs_dot_, JRys_dot_), (Jos_dot, JRxs_dot, JRys_dot)):
    for J in Js:
        newJs.append(J.subs([
        (sy.Derivative(q1(t),t), dq[0, 0]),
        (sy.Derivative(q2(t),t), dq[1, 0]),
        (sy.Derivative(q3(t),t), dq[2, 0]),
        (sy.Derivative(q4(t),t), dq[3, 0]),
        (q1(t), q[0, 0]),
        (q2(t), q[1, 0]),
        (q3(t), q[2, 0]),
        (q4(t), q[3, 0]),
    ]))


os = [sy.expand(e) for e in os]
Rxs = [sy.expand(e) for e in Rxs]
Rys = [sy.expand(e) for e in Rys]

Jos = [sy.expand(e) for e in Jos]
JRxs = [sy.expand(e) for e in JRxs]
JRys = [sy.expand(e) for e in JRys]

Jos_dot = [sy.expand(e) for e in Jos_dot]
JRxs_dot = [sy.expand(e) for e in JRxs_dot]
JRys_dot = [sy.expand(e) for e in JRys_dot]


expr_all = [os, Rxs, Rys, Jos, JRxs, JRys, Jos_dot, JRxs_dot, JRys_dot]
names = [str(i) for i in range(4)] + ["ee"]
expr_name = [
    ["o_" + n for n in names],
    ["rx_" + n for n in names],
    ["ry_" + n for n in names],
    ["jo_" + n for n in names],
    ["jrx_" + n for n in names],
    ["jry_" + n for n in names],
    ["jo_" + n + "_dot" for n in names],
    ["jrx_" + n + "_dot" for n in names],
    ["jry_" + n + "_dot" for n in names],
]

Matrix([[1, 0, 0], [0, 1, 0], [0, 0, 1]])
Matrix([[1, 0, l1], [0, 1, 0], [0, 0, 1]])
Matrix([[1, 0, l1 + l2], [0, 1, 0], [0, 0, 1]])
Matrix([[1, 0, l1 + l2 + l3], [0, 1, 0], [0, 0, 1]])
Matrix([[1, 0, l1 + l2 + l3 + l4], [0, 1, 0], [0, 0, 1]])


In [33]:
# Cコード
### これが本物 ###
from sympy.printing import cxxcode
from sympy.utilities.codegen import codegen
import os as OS


original = "cpp_original"
done = "cpp_done"

OS.makedirs(original, exist_ok=True)
OS.makedirs(original+"/include", exist_ok=True)
OS.makedirs(original+"/src", exist_ok=True)


def gen_cpp_code(expr, name, dir):
    [(c_name, c_code), (h_name, c_header)] = codegen(
        name_expr=(name, expr),
        language="C",
        project= name + "_BY_SYMPY_",
        to_files=False
    )
    
    f = open(dir+"/src/"+name+".cpp", 'w')
    f.write(c_code)
    f.close()

    f = open(dir+"/include/"+h_name.replace(".h", "")+".hpp", 'w')
    f.write(c_header)
    f.close()

for exprs, names in zip(expr_all, expr_name):
    for expr, name in zip(exprs, names):
        gen_cpp_code(expr, name, original)

com = "#ifndef FRANAK_EMIKA_HPP\n" \
    + "#define FRANKA_RMIKA_HPP\n" \
    + "#include<eigen3/Eigen/Core>\n" \
    + "#include<vector>\n"\
    + "namespace franka_emika\n" \
    + "{\n" \
    + "    using Eigen::VectorXd;\n" \
    + "    using Eigen::MatrixXd;\n" \
    + "using std::vector;\n"\
    + "using func_q_vecout = void (*)(const VectorXd&, VectorXd&);\n"\
    + "using func_q_matout = void (*)(const VectorXd&, MatrixXd&);\n"\
    + "using func_q_dq_matout = void (*)(const VectorXd&, const VectorXd&, MatrixXd&);\n"\
    + "\n"\
    + "    class Kinematics\n"\
    + "    {\n"\
    + "    private:\n"\
    + "        static const double d1;\n" \
    + "        static const double d3;\n" \
    + "        static const double d5;\n" \
    + "        static const double df;\n" \
    + "        static const double a4;\n" \
    + "        static const double a5;\n" \
    + "        static const double a7;\n" \

for ns in expr_name[0:4]:
    for n in ns:
        com += ("        static void " + n + "(const VectorXd& q, VectorXd& out);\n")
for ns in expr_name[4:8]:
    for n in ns:
        com += ("        static void " + n + "(const VectorXd& q, MatrixXd& out);\n")
for ns in expr_name[8:12]:
    for n in ns:
        com += ("        static void " + n + "(const VectorXd& q, const VectorXd& q_dot, MatrixXd& out);\n")

com += "    };\n};\n#endif"


### 変換 ###
import re
import os as OS
done = "cpp_done"
OS.makedirs(done, exist_ok=True)
OS.makedirs(done+"/include", exist_ok=True)
OS.makedirs(done+"/src", exist_ok=True)

pat = r'out_(.+?)\['
pat2 = r'out_(.+?)\)'
pat3 = r'\((.+?)\) {'
pat4 = r'#(.+?).h\"'

sout = ["out[" + str(i) + "]" for i in range(21)]
sout_2 = ["out(0,0)","out(0,1)","out(0,2)","out(0,3)","out(1,0)","out(1,1)","out(1,2)","out(1,3)"]

with open("cpp_done/include/sice.hpp", "w") as f:
    f.write(com)



def common_trans(line):
    r = re.findall(pat, line)
    r2 = re.findall(pat2, line)
    if len(r) != 0:
        line = line.replace("out_" + r[0], "out")
    if len(r2) != 0:
        line = line.replace("out_" + r2[0], "out")
    line = line.replace("q[0]", "q(0)")
    line = line.replace("q[1]", "q(1)")
    line = line.replace("q[2]", "q(2)")
    line = line.replace("q[3]", "q(3)")
    
    
    r3 = re.findall(pat3, line)
    if "j" not in name:
        if len(r3) != 0:
            print("("+r3[0]+")")
            #line = line.replace("("+r3[0]+") {", "(const VectorXd& q, VectorXd& out) {")
            line = line.replace("("+r3[0]+") {", "(const VectorXd& q, VectorXd& out) {")
        line = line.replace("double *out", "VectorXd& out")
        line = line.replace("out[0]", "out(0)")
        line = line.replace("out[1]", "out(1)")


    
    else:
        if "dot" in name:
            if len(r3) != 0:
                line = line.replace(r3[0], "const VectorXd& q, const VectorXd& dq, MatrixXd& out")
        else:
            if len(r3) != 0:
                print(name)
                line = line.replace(r3[0], "const VectorXd& q, MatrixXd& out")
        line = line.replace("double *out", "MatrixXd& out")
        for s, t in zip(sout, sout_2):
            line = line.replace(s, t)
        


    return line


def trans_cpp(name):
    origin = "cpp_original/src/" + name + ".cpp"
    done = "cpp_done/src/" + name + ".cpp"
    with open(origin, "r") as f, open(done, "w") as g:
        file_data = f.readlines()
        for line in file_data:
            line = line.replace('#include <math.h>', '#include <cmath>\nusing std::cos;\nusing std::sin;\nusing std::sqrt;\n')
            #line = line.replace("#include \"", "#include \"../../include/baxter/")
            #line = line.replace(".h\"", ".hpp\"\n#include \"../../include/baxter/common.hpp\"\n")
            r4 = re.findall(pat4, line)
            if len(r4) != 0:
                line = line.replace("#"+r4[0]+".h\"", "#include \"../include/sice.hpp\"\n")
            line = line.replace("void ", "void sice::Kinematics::")
            line = line.replace("double *q", "const VectorXd& q").replace("double *dq", "const VectorXd& dq")
            
            line = common_trans(line)


            g.write(line)


# def trans_hpp(name):
#     origin = "cpp_original/include/" + name + ".hpp"
#     done = "cpp_done/include/" + name + ".hpp"
#     with open(origin, "r") as f, open(done, "w") as g:
#         file_data = f.readlines()
#         for line in file_data:
#             line = line.replace("void ", "#include<eigen3/Eigen/Core>\nnamespace baxter\n{\nusing Eigen::VectorXd;\nusing Eigen::MatrixXd;\nvoid ").replace(");", ");\n}\n")
#             line = line.replace("double *q", "const VectorXd& q").replace("double *dq", "const VectorXd& dq")
            
#             line = common_trans(line)

#             g.write(line)

for names in expr_name:
    for name in names:
        trans_cpp(name)
        #trans_hpp(name)

(double *out)
(double l1, const VectorXd& q, double *out)
(double l1, double l2, const VectorXd& q, double *out)
(double l1, double l2, double l3, const VectorXd& q, double *out)
(double l1, double l2, double l3, double l4, const VectorXd& q, double *out)
(const VectorXd& q, double *out)
(const VectorXd& q, double *out)
(const VectorXd& q, double *out)
(const VectorXd& q, double *out)
(const VectorXd& q, double *out)
(const VectorXd& q, double *out)
(const VectorXd& q, double *out)
(const VectorXd& q, double *out)
(const VectorXd& q, double *out)
(const VectorXd& q, double *out)
jo_0
jo_1
jo_2
jo_3
jo_ee
jrx_0
jrx_1
jrx_2
jrx_3
jrx_ee
jry_0
jry_1
jry_2
jry_3
jry_ee


In [12]:
from sympy.printing.numpy import NumPyPrinter
import os as OS

names = [str(i) for i in range(4)] + ["ee"]

common_w = "import numpy as np\nfrom math import cos as c\nfrom math import sin as s\nfrom math import tan as ta\nfrom math import sqrt as sq\n"


dir_name = "py"
OS.makedirs(dir_name, exist_ok=True)

done_name = "done_py"
OS.makedirs(done_name, exist_ok=True)


with open(dir_name + "/htm.py", "w") as f:
    f.write(common_w)
    for name, z in zip(names, os):
        numpy_word = "def o_" + name + "(q):\n    return "
        #f.write(numba_word_q)
        f.write(numpy_word)
        f.write(NumPyPrinter().doprint(z))
        f.write("\n")

    for name, z in zip(names, Rxs):
        numpy_word = "def rx_" + name + "(q):\n    return "
        #f.write(numba_word_q)
        f.write(numpy_word)
        f.write(NumPyPrinter().doprint(z))
        f.write("\n")

    for name, z in zip(names, Rys):
        numpy_word = "def ry_" + name + "(q):\n    return "
        #f.write(numba_word_q)
        f.write(numpy_word)
        f.write(NumPyPrinter().doprint(z))
        f.write("\n")

    # for name, z in zip(names, Rzs):
    #     numpy_word = "def rz_" + name + "(q):\n    return "
    #     #f.write(numba_word_q)
    #     f.write(numpy_word)
    #     f.write(NumPyPrinter().doprint(z))
    #     f.write("\n")


with open(dir_name + "/Jos.py", "w") as f:
    f.write(common_w)
    for name, z in zip(names, Jos):
        numpy_word = "def jo_" + name + "(q):\n    return "
        #f.write(numba_word_q)
        f.write(numpy_word)
        f.write(NumPyPrinter().doprint(z))
        f.write("\n")

with open(dir_name + "/JRxs.py", "w") as f:
    f.write(common_w)
    for name, z in zip(names, JRxs):
        numpy_word = "def jrx_" + name + "(q):\n    return "
        #f.write(numba_word_q)
        f.write(numpy_word)
        f.write(NumPyPrinter().doprint(z))
        f.write("\n")

with open(dir_name + "/JRys.py", "w") as f:
    f.write(common_w)
    for name, z in zip(names, JRys):
        numpy_word = "def jry_" + name + "(q):\n    return "
        #f.write(numba_word_q)
        f.write(numpy_word)
        f.write(NumPyPrinter().doprint(z))
        f.write("\n")

# with open(dir_name + "/JRzs.py", "w") as f:
#     f.write(common_w)
#     for name, z in zip(names, JRzs):
#         numpy_word = "def jrz_" + name + "(q):\n    return "
#         #f.write(numba_word_q)
#         f.write(numpy_word)
#         f.write(NumPyPrinter().doprint(z))
#         f.write("\n")


with open(dir_name + "/Jo_dots.py", "w") as f:
    f.write(common_w)
    for name, z in zip(names, Jos_dot):
        numpy_word = "def jo_" + name + "_dot(q, dq):\n    return "
        #f.write(numba_word_q_dq)
        f.write(numpy_word)
        f.write(NumPyPrinter().doprint(sy.simplify(z)))
        f.write("\n")

with open(dir_name + "/JRx_dots.py", "w") as f:
    f.write(common_w)
    for name, z in zip(names, JRxs_dot):
        numpy_word = "def jrx_" + name + "_dot(q, dq):\n    return "
        #f.write(numba_word_q_dq)
        f.write(numpy_word)
        f.write(NumPyPrinter().doprint(sy.simplify(z)))
        f.write("\n")

with open(dir_name + "/JRy_dots.py", "w") as f:
    f.write(common_w)
    for name, z in zip(names, JRys):
        numpy_word = "def jry_" + name + "_dot(q, dq):\n    return "
        #f.write(numba_word_q_dq)
        f.write(numpy_word)
        f.write(NumPyPrinter().doprint(sy.simplify(z)))
        f.write("\n")

# with open(dir_name + "/JRz_dots.py", "w") as f:
#     f.write(common_w)
#     for name, z in zip(names, JRzs):
#         numpy_word = "def jrz_" + name + "_dot(q, dq):\n    return "
#         #f.write(numba_word_q_dq)
#         f.write(numpy_word)
#         f.write(NumPyPrinter().doprint(sy.simplify(z)))
#         f.write("\n")


def translate_hoge(original, done):
    with open(original, "r") as f, open(done, "w") as g:
        file_data = f.readlines()
        for line in file_data:
            line = line.replace('numpy', 'np').replace('1/2', '0.5').replace('(0.5)', '0.5')
            line = line.replace('np.cos', 'c').replace('np.sin', 's').replace('np.sqrt', 'sq')
            line = line.replace('l1', "cls.l1").replace('l2', "cls.l2").replace('l3', "cls.l3").replace('l4', "cls.l4")
            line = line.replace('(q)', '(cls, q)').replace('(q, dq)', '(cls, q, dq)')
            line = line.replace('def', '@classmethod\ndef')
            line = line.replace('import np as np', 'import numpy as np')
            g.write(line)



translate_hoge(dir_name + "/htm.py", done_name + "/htm.py")
translate_hoge(dir_name + "/Jos.py", done_name + "/Jos.py")
translate_hoge(dir_name + "/JRxs.py", done_name + "/JRxs.py")
translate_hoge(dir_name + "/JRys.py", done_name + "/JRys.py")
#translate_hoge(dir_name + "/JRzs.py", done_name + "/JRzs.py")
translate_hoge(dir_name + "/Jo_dots.py", done_name + "/Jo_dots.py")
translate_hoge(dir_name + "/JRx_dots.py", done_name + "/JRx_dots.py")
translate_hoge(dir_name + "/JRy_dots.py", done_name + "/JRy_dots.py")
#translate_hoge(dir_name + "/JRz_dots.py", done_name + "/JRz_dots.py")